这是一个使用CodeFuse-Query分析java项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用Godel语言来分析这个仓库。

STEP 0: 克隆要分析的仓库。我们以蚂蚁集团的 SOFABolt 项目为例。

In [1]:
!git clone https://gitee.com/sofastack/sofa-bolt.git

Cloning into 'sofa-bolt'...
remote: Enumerating objects: 3378, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 3378 (delta 69), reused 43 (delta 6), pack-reused 3149
Receiving objects: 100% (3378/3378), 1.03 MiB | 447.00 KiB/s, done.
Resolving deltas: 100% (1794/1794), done.
Updating files: 100% (325/325), done.


STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，需要指定待分析的仓库地址，分析的语言，以及db文件的存储路径。该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行。

In [2]:
!sparrow database create --source-root sofa-bolt --data-language-type java --output ./db/sofa-bolt

Missing required option: '--data-type=<type>'

Usage:

sparrow database create [-C=<file>] [-log=<level>] [-s=<dir>] [-c=<item>]...
                        -t=<type>[,<type>...] [-t=<type>[,<type>...]]...
                        <database>

Description:

Create a COREF database for a source tree that can be analyzed by sparrow

Parameters:

      <database>            Path to the COREF database to create. This
                              directory will be created, and must not already
                              exist

Options:

  -c, --extraction-config=<item>
                            Extraction config, -c c1=xx1 -c c2=xx2,xx3
                             Examples of how config items are parsed:
                             1.java.cp=/Users/xxx/.
                              m2/repository/com/google/code/gson/gson/2.
                              10/gson-2.10.jar
                             cp=/Users/xxx/.
                              m2/repository/com/google/code/gson/gson/

STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，直接运行分析脚本。在命令行中，你可以使用 `sparrow query run` 命令来执行查询脚本，具体可以使用`sparrow query run -h` 来获取详细参数信息。

In [2]:
!export PATH=/Users/zhou/.sdkman/candidates/maven/current/bin:/Users/zhou/.sdkman/candidates/java/current/bin:/Users/zhou/.nvm/versions/node/v20.8.1/bin:/Users/zhou/miniconda3/bin:/Users/zhou/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin:/Users/zhou/Library/Application\ Support/JetBrains/Toolbox/scripts:/Users/zhou/Documents/project/sparrow-cli-dist/sparrow-cli

In [9]:
!which sparrow

/Users/zhou/Documents/project/sparrow-cli-dist/sparrow-cli/sparrow


In [ ]:
%db db/sofa-bolt
// script
use coref::java::{Callable, Class, Interface, JavaDB}

fn db() -> JavaDB {
    return JavaDB::load("coref_java_src.db")
}

query class_method from
    m in Callable(db()),
    c in Class(db())
where
    c.id = m.getBelongedClass().id
select
    c.getQualifiedName() as className,
    m.getName() as methodName,
    m.getSignature() as methodSignature


query interface_method from
    m in Callable(db()),
    i in Interface(db())
where
    i = m.getBelongedInterface()
select
    i.getQualifiedName() as className,
    m.getName() as methodName,
    m.getSignature() as methodSignature
